In [1]:
using Lathe.models: Pipeline, LatheObject

In [2]:
mutable struct Router{P} <: LatheObject
    fn::Function
    components::Array{LatheObject}
    predict::P
    function Router(components::LatheObject ...; fn::Function)
        components = Array([comp for comp in components])
        predict(xt) = router_load(xt, fn, components)
        new{typeof(predict)}(fn, components, predict)
    end
    function router_load(data, fn, components)
        returns = fn(data)
        preds = []
        count = 0
        for cp in components
            count += 1
            res = cp.predict(returns[count])
            push!(preds, res)
        end
        return([res for res in preds])
    end
end

In [3]:
using Lathe.preprocess: OrdinalEncoder, FloatEncoder

In [4]:
x = ["Hi", "hello"]

2-element Vector{String}:
 "Hi"
 "hello"

In [5]:
pipe1 = Pipeline(OrdinalEncoder(x))
pipe2 = Pipeline(FloatEncoder())

Pipeline{Lathe.models.var"#predict#54"{Vector{FloatEncoder{Lathe.preprocess.var"#predict#12"}}}, Lathe.models.var"#show#53"{Vector{FloatEncoder{Lathe.preprocess.var"#predict#12"}}}}(LatheObject[FloatEncoder{Lathe.preprocess.var"#predict#12"}(Lathe.preprocess.var"#predict#12"())], Lathe.models.var"#predict#54"{Vector{FloatEncoder{Lathe.preprocess.var"#predict#12"}}}(FloatEncoder{Lathe.preprocess.var"#predict#12"}[FloatEncoder{Lathe.preprocess.var"#predict#12"}(Lathe.preprocess.var"#predict#12"())], Core.Box(Lathe.models.var"#pipe_predict#56"())), Lathe.models.var"#show#53"{Vector{FloatEncoder{Lathe.preprocess.var"#predict#12"}}}(FloatEncoder{Lathe.preprocess.var"#predict#12"}[FloatEncoder{Lathe.preprocess.var"#predict#12"}(Lathe.preprocess.var"#predict#12"())], Core.Box(Lathe.models.var"#_show#55"())))

In [6]:
function take_x(xdata)
    return(xdata, xdata)
end

take_x (generic function with 1 method)

In [7]:
router = Router(pipe1, pipe2, fn = take_x)

Router{var"#predict#3"{typeof(take_x), Vector{Pipeline}}}(take_x, LatheObject[Pipeline{Lathe.models.var"#predict#54"{Vector{OrdinalEncoder{Lathe.preprocess.var"#predict#9"{Dict{String, Int64}}}}}, Lathe.models.var"#show#53"{Vector{OrdinalEncoder{Lathe.preprocess.var"#predict#9"{Dict{String, Int64}}}}}}(LatheObject[OrdinalEncoder{Lathe.preprocess.var"#predict#9"{Dict{String, Int64}}}(Lathe.preprocess.var"#predict#9"{Dict{String, Int64}}(Dict("Hi" => 1, "hello" => 2)), Dict("Hi" => 1, "hello" => 2))], Lathe.models.var"#predict#54"{Vector{OrdinalEncoder{Lathe.preprocess.var"#predict#9"{Dict{String, Int64}}}}}(OrdinalEncoder{Lathe.preprocess.var"#predict#9"{Dict{String, Int64}}}[OrdinalEncoder{Lathe.preprocess.var"#predict#9"{Dict{String, Int64}}}(Lathe.preprocess.var"#predict#9"{Dict{String, Int64}}(Dict("Hi" => 1, "hello" => 2)), Dict("Hi" => 1, "hello" => 2))], Core.Box(Lathe.models.var"#pipe_predict#56"())), Lathe.models.var"#show#53"{Vector{OrdinalEncoder{Lathe.preprocess.var"#predict

In [8]:
router.predict(x)

2-element Vector{Vector{T} where T}:
 [1, 2]
 Any[177.0, 532.0]

In [9]:
using Lathe.stats: mean

In [10]:
z = missing

missing

In [11]:
typeof(z)

Missing

In [12]:
b = NaN

NaN

In [86]:
abstract type Filler <: LatheObject end
mutable struct MeanFiller{P} <: Filler
    mu::Float64
    predict::P
    function MeanFiller(data::Array)
        mu = mean_missings(data)
        predict(xt::Array) = fill_null(xt, mu)
        new{typeof(predict)}(mu, predict)
    end
    function fill_null(xt::Array, mu::Float64)
        z = []
        for i in xt
            if ismissing(i)
                newi = mu
            elseif typeof(i) == String
                newi = mu
            elseif isnan(i)
                newi = mu
            else
                newi = i
            end
            append!(z, newi)
        end
        return(z)
    end
    function mean_missings(x::Array)
        total = 0
        len = 0
        for value in x
            if ismissing(value)
            elseif typeof(value) == String
            elseif isnan(value)
            else
                total += value
                len += 1
            end
        end
        return(total / len)
    end
end

LoadError: invalid redefinition of constant MeanFiller

In [87]:
sample_data = [5, 10, 15, missing, 25, NaN, 14, 22, "Hi"]

9-element Vector{Any}:
   5
  10
  15
    missing
  25
 NaN
  14
  22
    "Hi"

In [82]:
if isnan(sample_data[6])
    println("yes")
end

yes


In [83]:
mf = MeanFiller(sample_data)

MeanFiller{var"#predict#43"{Float64}}(15.166666666666666, var"#predict#43"{Float64}(15.166666666666666, Core.Box(var"#fill_null#44"())))

In [84]:
mf.mu

15.166666666666666

In [85]:
new_data = mf.predict(sample_data)

9-element Vector{Any}:
  5
 10
 15
 15.166666666666666
 25
 15.166666666666666
 14
 22
 15.166666666666666

In [89]:
using Lathe.models: ClassBaseline
mutable struct ClassFiller{P} <: Filler
    m::LatheObject
    predict::P
    function ClassFiller(x::Array)
        m = ClassBaseline(x)
        predict(x::Array) = nan_pred(m, x)
        new{typeof(predict)}(m, predict)
    end
    function nan_pred(m::LatheObject, x::Array)
        newx = []
        for dim in x
            if ismissing(dim)
                n = m.predict([value])[1] # The output comes in the form of
            elseif isnan(dim)           #    an array, so we must call [1]
                n = m.predict([value])[1]
            else
                n = n
            end
            append!(newx, n)
        end
        return(newx)
    end
end
    

In [90]:
x = ["H", "B", "H", missing, "H", NaN]

6-element Vector{Any}:
    "H"
    "B"
    "H"
    missing
    "H"
 NaN

In [91]:
d = ClassFiller(x)

LoadError: TypeError: non-boolean (Missing) used in boolean context